In [9]:
from flask import Flask, render_template, url_for, request 
from PIL import Image 
import tensorflow as tf
import numpy as np 
import glob 
from numpy import array
from sklearn.preprocessing import LabelEncoder 
import pandas as pd
from werkzeug import secure_filename
from sklearn.metrics import accuracy_score

hidden=1000
learning_rate=0.01
epochs=1000

seed1=350
seed2=750

width=5 
height=5
channels=3

#I obtain a better accuracy when I set a high number of neurons and a low width and height size when resizing the images



In [10]:

#flatten list of list of lists
def put_list_of_lists_of_lists_into_single_list(lista): 
    lista_unfolded=[] 
    for sublist_uno in array(lista):
        for sublist_dos in sublist_uno:
            for sublist_tres in sublist_dos:
                lista_unfolded.append(sublist_tres)        
    return lista_unfolded

#define weights connecting initial and hidden layers
def init_weights(shape): 
    weights = tf.random_normal(shape, stddev=0.1,seed=seed1) 
    return tf.Variable(weights) 
    #return weights

#method to set biases for middle layer
def init_biases(shape):  
	biases = tf.random_normal(shape, stddev=0.1,seed=seed2) 
	return tf.Variable(biases)     

#method to perform forwarding
def forwardprop(X, w_1, w_2): 
    h    = tf.nn.sigmoid(tf.matmul( X,w_1)+b_1)
    yhat = tf.sigmoid(tf.matmul(h,w_2))
    return yhat
    

In [ ]:
#read train and test images. Then resize

imagesTrain=[]
number_images_train=len(glob.glob('D:/classifier/tatto_train/*.jpg'))
number_images_test=len(glob.glob('D:/classifier/tatto_test/*.jpg'))
print(number_images_train)
print(number_images_test)
for filename in glob.glob('D:/classifier/tatto_train/*.jpg'):
	image=Image.open(filename)
	image_resized=image.resize((width,height)) 
	imagesTrain.append(image_resized)

imagesTest=[]   
number_images_test=len(glob.glob('D:/classifier/tatto_test/*.jpg'))
for filename in glob.glob('D:/classifier/tatto_test/*.jpg'):
	image=Image.open(filename)
	image_resized=image.resize((width,height)) 
	imagesTest.append(image_resized)    
    

In [ ]:
#reshape 

imagesTrainUnfolded=map(put_list_of_lists_of_lists_into_single_list,imagesTrain) 
imagesTrainUnfolded=np.array(list(imagesTrainUnfolded)) 
imagesTrainUnfolded=np.float32(imagesTrainUnfolded)
imagesTrainUnfolded=imagesTrainUnfolded.reshape(number_images_train,1,total)

imagesTestUnfolded=map(put_list_of_lists_of_lists_into_single_list,imagesTest) 
imagesTestUnfolded=np.array(list(imagesTestUnfolded)) 
imagesTestUnfolded=np.float32(imagesTestUnfolded)
imagesTestUnfolded=imagesTestUnfolded.reshape(number_images_test,1,total) 

In [12]:
#label encoder

imagesNamesTrain=glob.glob('D:/classifier/tatto_train/*.jpg')
labelsTrain=[i.replace('D:/classifier/tatto_train\\','') for i in imagesNamesTrain]

imagesNamesTest=glob.glob('D:/classifier/tatto_test/*.jpg')
labelsTest=[i.replace('D:/classifier/tatto_test\\','') for i in imagesNamesTest]

labelsTrain=[int(i[0]) for i in labelsTrain]
labelsTest=[int(i[0]) for i in labelsTest]

#label encoder (0 is skull, 1 is woman, 2 is monster)
le=LabelEncoder() 
le.fit_transform(['calavera','mujer','monstruo']) 

labelsTrain=np.eye(3)[labelsTrain].reshape(number_images_train,1,3) 

train_X=imagesTrainUnfolded 
train_y=labelsTrain

test_X=imagesTestUnfolded
test_y=labelsTest
    

In [15]:

tf.reset_default_graph()
X=tf.placeholder(tf.float32,shape=[None,1,total]) 
y=tf.placeholder(tf.float32,shape=[None,1,3])

total=width*height*channels

#input size, which is the multiplication of width times height times channels
x_size=total 
#output size
y_size=3 
#number of neurons contained in middle layer
h_size=hidden 
#weights connecting initial layer with middle layer
w_1=init_weights((x_size,h_size)) 
w_1=tf.tile(tf.expand_dims(w_1,axis=0),[number_images_train,1,1])
#weights connecting middle layer with final layer
w_2=init_weights((h_size,y_size)) 
w_2=tf.tile(tf.expand_dims(w_2,axis=0),[number_images_train,1,1])

#set biases values for middle layer
b_1=init_biases((1,h_size)) 
b_1=tf.tile(tf.expand_dims(b_1,axis=0),[number_images_train,1,1])

yhat = forwardprop(X, w_1, w_2) 

predict = tf.argmax(yhat, axis=2) 

cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=yhat)) 

updates=tf.train.GradientDescentOptimizer(learning_rate).minimize(cost) 

predicciones_test=[]
init=tf.global_variables_initializer() 
with tf.Session() as sess:
    sess.run(init) 
    for epoch in range(epochs):
        sess.run(updates,feed_dict={X:train_X,y:train_y}) 
    W_1=sess.run(w_1)[0,:,:]
    W_2=sess.run(w_2)[0,:,:]
    B_1=sess.run(b_1)[0,:,:]
    for i in test_X:
        first_layer = tf.nn.sigmoid(tf.matmul(i,W_1)+B_1)
        YHAT = tf.argmax(tf.sigmoid(tf.matmul(first_layer,W_2)),axis=1)
        prediccion=sess.run(YHAT)
        print(prediccion)
        predicciones_test.append(prediccion)

print(W_1)
print(W_2)
print(B_1)
print(accuracy_score(np.array(predicciones_test).reshape(-1),labelsTest))


[0]
[0]
[2]
[0]
[1]
[1]
[2]
[1]
[0]
[1]
[2]
[0]
[1]
[0]
[2]
[0]
[0]
[2]
[0]
[0]
[1]
[2]
[2]
[0]
[0]
[0]
[2]
[0]
[0]
[1]
[2]
[0]
[2]
[0]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[2]
[2]
[2]
[2]
[1]
[1]
[0]
[1]
[2]
[0]
[1]
[1]
[1]
[0]
[1]
[0]
[2]
[2]
[0]
[0]
[2]
[1]
[0]
[0]
[0]
[0]
[1]
[1]
[0]
[0]
[1]
[2]
[2]
[2]
[2]
[2]
[1]
[0]
[2]
[1]
[2]
[2]
[0]
[2]
[0]
[2]
[0]
[2]
[[ 0.03388737 -0.07293987 -0.09090511 ... -0.12844107 -0.10440452
   0.11772938]
 [-0.13999599 -0.04147361  0.11535463 ...  0.17580098  0.05918317
   0.04679963]
 [ 0.12177155  0.04002747 -0.02940186 ...  0.01297749 -0.06042715
   0.00684824]
 ...
 [ 0.08819972 -0.14160557 -0.11304628 ... -0.1868365  -0.08245976
   0.02244789]
 [-0.1596655  -0.01415089  0.10778493 ...  0.06915473  0.04163013
   0.09253891]
 [ 0.07706357 -0.02818604 -0.00052762 ...  0.02063455 -0.01567096
   0.01650061]]
[[ 0.01943577 -0.07912252 -0.08910513]
 [-0.08452232  0.125045    0.00591138]
 [ 0.09668005 -0.13973515  0.08002353]
 ...
 [ 0.16626322 -0.04818994 

In [14]:
#pickle weights and biases
import pickle
myVariables=[W_1,W_2,B_1,width,height,total]
pickling=open("D:/classifier/myVariables.pickle","wb")
pickle.dump(myVariables,pickling)
pickling.close()